# Conversational AI with Structured Pydantic Extraction

Testing three approaches to having a natural conversation and extracting structured Pydantic fields at the end — similar to how Rasa handles form-filling.

## Setup

In [7]:
# !pip install openai pydantic instructor dotenv

In [4]:
import os
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional

# Set your API key (or set OPENAI_API_KEY env var before starting jupyter)
# os.environ["OPENAI_API_KEY"] = "sk-..."
# import from .env

from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

## Define the Pydantic Model

This is the structured data we want to extract after a natural conversation.

In [5]:
class CustomerRequest(BaseModel):
    """Structured data extracted from a customer conversation."""
    customer_name: str = Field(description="Customer's full name")
    email: str = Field(description="Customer's email address")
    issue_category: str = Field(description="Category: billing, technical, account, or other")
    issue_description: str = Field(description="Summary of the customer's issue")
    urgency: str = Field(description="low, medium, or high")
    resolution_requested: Optional[str] = Field(default=None, description="What the customer wants done")

---
## Approach 1: OpenAI Structured Outputs (Native)

Have a multi-turn conversation, then parse the final extraction into a Pydantic model using `response_format`.

In [4]:
SYSTEM_PROMPT = """You are a friendly customer support agent.
Have a natural conversation with the customer to understand their issue.
Ask follow-up questions naturally until you have:
- Their name
- Their email
- What category the issue falls into (billing, technical, account, or other)
- A clear description of the issue
- How urgent it is
- What resolution they'd like

Be conversational and empathetic. Don't ask for all info at once — 
gather it naturally over the conversation."""

messages = [{"role": "system", "content": SYSTEM_PROMPT}]

In [18]:
def chat(user_message: str) -> str:
    """Send a message and get a response. Run this cell multiple times to have a conversation."""
    messages.append({"role": "user", "content": user_message})
    
    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=messages,
    )
    reply = response.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    return reply

In [19]:
# Start the conversation — edit and re-run this cell to chat back and forth
print(chat("Hi, I'm having a problem with my account"))

Sorry you’re running into trouble — I’m happy to help. Could I get your name first, and a brief description of what’s happening with your account (for example: can’t sign in, locked out, incorrect billing, missing access, etc.)?


In [20]:
# Continue chatting (edit the message each time)
print(chat("My name is John Smith, email john@example.com. I got double-charged on my last bill and it's pretty urgent."))

Thanks, John — I’m sorry that happened and I’ll get this sorted for you.

I see the issue is billing-related and you said it’s pretty urgent. Before I start an investigation, a couple quick questions so I can escalate this right away:

- Can you tell me the date of the charge(s) and the exact amounts shown on your card/bank statement?  
- Do you have any invoice or transaction IDs (from your account billing history or your bank) or a screenshot you can attach?  
- What payment method was used (card type) and can you share the last 4 digits of the card? (This helps me match the transactions — don’t send the full card number.)  
- Is the duplicate charge showing on your bank/credit card statement as two separate postings, or is one a pending/authorization?  
- What outcome would you prefer: an immediate refund to your card, an account credit, or an explanation and confirmation that the extra charge will be reversed?  
- Any other details I should know (e.g., invoice number, subscription 

In [21]:
print(chat("I'd like a refund for the duplicate charge please."))

Thanks, John — I’m really sorry this happened and I’ll get it taken care of right away.

I can start a refund for the duplicate charge to your original payment method. A couple quick details I need to locate the exact transactions and process the refund:

- Date(s) of the charge(s) and the exact amounts shown on your statement  
- Last 4 digits of the card used (please don’t send the full card number)  
- Are both charges posted on your bank/credit-card statement, or is one still pending/authorization?  
- Any invoice or transaction ID from your billing history (or a screenshot) if you have it

If you don’t have all of that handy, that’s okay — I can search your account (John Smith / john@example.com) with the date and last 4 digits. Please confirm you want the refund returned to the original card (is that correct?), and that I should proceed now. I’ll email updates to john@example.com.

Timeline: I’ll escalate this to our billing team immediately. Once we issue the refund it typically

In [22]:
# When ready, extract structured data from the conversation
extraction_messages = messages + [
    {"role": "user", "content": "Please extract the structured customer request from this conversation."}
]

response = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=extraction_messages,
    response_format=CustomerRequest,
)

result = response.choices[0].message.parsed
print("Extracted Pydantic object:")
print(result)
print()
print("As dict:")
print(result.model_dump())

Extracted Pydantic object:
customer_name='John Smith' email='john@example.com' issue_category='billing' issue_description='Double-charged on the last bill.' urgency='high' resolution_requested='Refund for the duplicate charge.'

As dict:
{'customer_name': 'John Smith', 'email': 'john@example.com', 'issue_category': 'billing', 'issue_description': 'Double-charged on the last bill.', 'urgency': 'high', 'resolution_requested': 'Refund for the duplicate charge.'}


---
## Approach 2: Instructor Library

Uses function calling under the hood with automatic retries and validation.

In [5]:
import instructor

instructor_client = instructor.from_openai(OpenAI())

In [7]:
messages

[{'role': 'system',
  'content': "You are a friendly customer support agent.\nHave a natural conversation with the customer to understand their issue.\nAsk follow-up questions naturally until you have:\n- Their name\n- Their email\n- What category the issue falls into (billing, technical, account, or other)\n- A clear description of the issue\n- How urgent it is\n- What resolution they'd like\n\nBe conversational and empathetic. Don't ask for all info at once — \ngather it naturally over the conversation."}]

In [8]:
# Extract from the same conversation history we built above
result_instructor = instructor_client.chat.completions.create(
    model="gpt-5-mini",
    response_model=CustomerRequest,
    messages=messages,
    max_retries=2,  # auto-retries if validation fails
)

print("Instructor extraction:")
print(result_instructor)
print()
print("As dict:")
print(result_instructor.model_dump())

Instructor extraction:
customer_name='' email='' issue_category='other' issue_description='' urgency='medium' resolution_requested=None

As dict:
{'customer_name': '', 'email': '', 'issue_category': 'other', 'issue_description': '', 'urgency': 'medium', 'resolution_requested': None}


---
## Approach 3: Rasa-Style Conversational Loop

The AI chats naturally, decides when it has enough info, and signals completion. This is the closest to Rasa's form-filling behavior.

In [6]:
import json

LOOP_SYSTEM_PROMPT = """You are a friendly customer support agent.
Have a natural conversation to collect the following information:
- Customer's full name
- Email address
- Issue category (billing, technical, account, or other)
- Description of their issue
- Urgency (low, medium, high)
- What resolution they want

RULES:
1. Be conversational and empathetic. Don't interrogate — chat naturally.
2. Ask for missing info with follow-up questions, one or two at a time.
3. When you have ALL required info, respond with EXACTLY this format:

[COMPLETE]
{"customer_name": "...", "email": "...", "issue_category": "...", "issue_description": "...", "urgency": "...", "resolution_requested": "..."}

4. Do NOT output [COMPLETE] until you have every field.
5. Before outputting [COMPLETE], confirm the details with the customer."""


def run_rasa_style_loop():
    """Interactive loop — type your messages, the AI chats back until it extracts all fields."""
    loop_messages = [{"role": "system", "content": LOOP_SYSTEM_PROMPT}]

    # Get initial greeting
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=loop_messages + [{"role": "user", "content": "Hi"}],
    )
    greeting = response.choices[0].message.content
    loop_messages.append({"role": "user", "content": "Hi"})
    loop_messages.append({"role": "assistant", "content": greeting})
    print(f"Agent: {greeting}\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ("quit", "exit"):
            print("Conversation ended by user.")
            return None

        loop_messages.append({"role": "user", "content": user_input})

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=loop_messages,
        )
        reply = response.choices[0].message.content
        loop_messages.append({"role": "assistant", "content": reply})

        if "[COMPLETE]" in reply:
            # Extract JSON after the marker
            json_str = reply.split("[COMPLETE]")[1].strip()
            # Show the confirmation text before [COMPLETE] if any
            before = reply.split("[COMPLETE]")[0].strip()
            if before:
                print(f"Agent: {before}\n")

            data = json.loads(json_str)
            result = CustomerRequest(**data)
            print("--- Extraction Complete ---")
            print(result.model_dump_json(indent=2))
            return result
        else:
            print(f"Agent: {reply}\n")

In [7]:
# Run the interactive loop (type 'quit' to exit early)
extracted = run_rasa_style_loop()

Agent: Hello there! How can I assist you today?

Agent: It seems like your message didn't come through. How can I help you today?

Agent: Oh no, I'm sorry to hear that! Could you tell me a little more about the problem you're experiencing?

Agent: Hi again! What's going on? How can I assist you with your problem today?

Agent: I see, you'd like a refund. Let's get that sorted out for you. Could you please tell me your full name and email address so I can locate your account?

Agent: It seems like your last message didn't come through. Could you share your full name and email address so I can help with your refund?



KeyboardInterrupt: Interrupted by user

In [3]:
# Inspect the result
if extracted:
    print(f"Name:       {extracted.customer_name}")
    print(f"Email:      {extracted.email}")
    print(f"Category:   {extracted.issue_category}")
    print(f"Issue:      {extracted.issue_description}")
    print(f"Urgency:    {extracted.urgency}")
    print(f"Resolution: {extracted.resolution_requested}")

NameError: name 'extracted' is not defined

---
## Approach Comparison

| Approach | Pros | Cons |
|---|---|---|
| **Structured Outputs** | Native OpenAI, guaranteed schema | Extraction is a separate call at the end |
| **Instructor** | Auto-retries, validation, clean API | Extra dependency |
| **Rasa-style Loop** | AI decides when info is complete, most natural | Needs prompt engineering, JSON parsing can fail |

For production, consider combining: use the **Rasa-style loop** for the conversation, then use **Instructor** or **Structured Outputs** for the final extraction (more reliable than parsing `[COMPLETE]` markers).

---
## Bonus: Hybrid Approach (Recommended for Production)

Chat naturally, then use structured outputs for reliable extraction.

In [8]:
HYBRID_SYSTEM = """You are a friendly customer support agent.
Have a natural conversation to understand the customer's issue.
Collect: name, email, issue category, description, urgency, desired resolution.
Be conversational. Don't interrogate.
When you believe you have all info, say: "I think I have everything I need. Let me summarize..."
and provide a natural-language summary for the customer to confirm."""


def run_hybrid_loop():
    hybrid_messages = [{"role": "system", "content": HYBRID_SYSTEM}]

    # Initial greeting
    hybrid_messages.append({"role": "user", "content": "Hello"})
    response = client.chat.completions.create(model="gpt-4o", messages=hybrid_messages)
    reply = response.choices[0].message.content
    hybrid_messages.append({"role": "assistant", "content": reply})
    print(f"Agent: {reply}\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ("quit", "exit"):
            return None

        hybrid_messages.append({"role": "user", "content": user_input})
        response = client.chat.completions.create(model="gpt-4o", messages=hybrid_messages)
        reply = response.choices[0].message.content
        hybrid_messages.append({"role": "assistant", "content": reply})
        print(f"Agent: {reply}\n")

        # Check if agent thinks it has everything
        if "everything i need" in reply.lower() or "let me summarize" in reply.lower():
            confirm = input("You (confirm or correct): ")
            hybrid_messages.append({"role": "user", "content": confirm})

            if any(w in confirm.lower() for w in ["yes", "correct", "looks good", "confirmed", "that's right"]):
                # Use structured outputs for reliable extraction
                print("\nExtracting structured data...\n")
                extraction = client.beta.chat.completions.parse(
                    model="gpt-4o",
                    messages=hybrid_messages,
                    response_format=CustomerRequest,
                )
                result = extraction.choices[0].message.parsed
                print("--- Extraction Complete ---")
                print(result.model_dump_json(indent=2))
                return result
            else:
                # Continue conversation to collect corrections
                response = client.chat.completions.create(model="gpt-4o", messages=hybrid_messages)
                reply = response.choices[0].message.content
                hybrid_messages.append({"role": "assistant", "content": reply})
                print(f"Agent: {reply}\n")

In [9]:
# Run the hybrid loop
hybrid_result = run_hybrid_loop()

Agent: Hi there! How can I help you today?

Agent: Hey! What's going on? How can I assist you today?

Agent: I'm sorry to hear that. Could you tell me a bit more about the problem you're experiencing? I'm here to help!

Agent: Got it. Refunds are important, and I'd be happy to help with that. Could you tell me a little more about the situation? Like what the product or service was, and if there are any specific issues you encountered?

Agent: Oh no, that sounds frustrating. I can definitely see why you'd want to get that fixed quickly. Could you let me know your name and email so I can look into your account and get this resolved for you?

Agent: Thanks, Wael. Just to clarify, the email is "wamansou@asu.edu," right? And to make sure we're on the same page, you're looking for a refund because you were double-charged on your last bill. Since you mentioned it's pretty urgent, we're aiming to resolve this as quickly as possible. Is there anything else you'd like to add or a specific timeli

In [10]:
if hybrid_result:
    print(type(hybrid_result))  # <class 'CustomerRequest'>
    print(hybrid_result.model_dump())

<class '__main__.CustomerRequest'>
{'customer_name': 'Wael', 'email': 'wamansou@asu.edu', 'issue_category': 'billing', 'issue_description': 'Double-charged on the last bill.', 'urgency': 'high', 'resolution_requested': 'Refund for the double charge.'}


---
## Approach 5: OpenAI Responses API (Newer than Chat Completions)

The Responses API (`client.responses.parse()`) is OpenAI's newer extraction API. It uses `text_format=` instead of `response_format=` and returns `response.output_parsed`.

In [11]:
# --- Responses API: Extract from a conversation ---

# Build a conversation first (reusing our messages from earlier, or start fresh)
conversation = [
    {"role": "system", "content": "You are a customer support agent. Extract structured info from conversations."},
    {"role": "user", "content": "Hi, I'm Jane Doe, jane@company.com. My software keeps crashing when I try to export reports. It's blocking my whole team."},
    {"role": "assistant", "content": "I'm sorry to hear that, Jane. That sounds really frustrating. Can you tell me which version you're on and when this started happening?"},
    {"role": "user", "content": "Version 3.2.1, started yesterday after the update. We need this fixed ASAP — it's high priority. Ideally roll back the update or patch the export."},
]

# Use the new Responses API
response = client.responses.parse(
    model="gpt-5-mini",
    input=conversation,
    text_format=CustomerRequest,
)

result_responses_api = response.output_parsed
print("Responses API extraction:")
print(result_responses_api.model_dump_json(indent=2))

Responses API extraction:
{
  "customer_name": "Jane Doe",
  "email": "jane@company.com",
  "issue_category": "technical",
  "issue_description": "Software crashes when exporting reports after updating to version 3.2.1; started yesterday and is blocking the team.",
  "urgency": "high",
  "resolution_requested": "Roll back the update or patch the export."
}


---
## Approach 6: OpenAI Agents SDK — Multi-Agent Triage (Most Rasa-Like)

This is the **most advanced** and **closest to Rasa**. The OpenAI Agents SDK provides:

- **Triage Agent** → routes customers to specialists (like Rasa's intent routing / stories)
- **Specialist Agents** with handoffs (like Rasa's forms)
- **Structured `output_type`** → Pydantic models as agent output (like Rasa's slots)
- **SQLiteSession** → automatic multi-turn conversation memory (like Rasa's tracker store)
- **Guardrails** → input validation before processing

### Architecture:
```
Customer → Triage Agent → Billing Agent (output_type=BillingTicket)
                        → Technical Agent (output_type=TechnicalTicket)
                        → Account Agent (output_type=AccountTicket)
```

In [12]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 10.5 MB/s  0:00:00 10.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [openai-agents]━━ 18/19 [openai-agents]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [13]:
import asyncio
from pydantic import BaseModel, Field
from typing import Optional
from agents import Agent, Runner, handoff, RunContextWrapper, InputGuardrail, GuardrailFunctionOutput
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX


# ──────────────────────────────────────────────
# 1. Define structured output models (like Rasa slots)
# ──────────────────────────────────────────────

class BillingTicket(BaseModel):
    """Structured output for billing issues."""
    customer_name: str = Field(description="Customer's full name")
    email: str = Field(description="Customer's email")
    charge_amount: Optional[float] = Field(default=None, description="Disputed charge amount")
    billing_issue: str = Field(description="Description of the billing problem")
    resolution: str = Field(description="Requested resolution: refund, credit, or investigation")
    urgency: str = Field(description="low, medium, or high")


class TechnicalTicket(BaseModel):
    """Structured output for technical issues."""
    customer_name: str = Field(description="Customer's full name")
    email: str = Field(description="Customer's email")
    product_version: Optional[str] = Field(default=None, description="Software/product version")
    error_description: str = Field(description="What's going wrong")
    steps_to_reproduce: Optional[str] = Field(default=None, description="How to reproduce the issue")
    urgency: str = Field(description="low, medium, or high")


class AccountTicket(BaseModel):
    """Structured output for account issues."""
    customer_name: str = Field(description="Customer's full name")
    email: str = Field(description="Customer's email")
    account_issue: str = Field(description="Description of the account problem")
    action_requested: str = Field(description="What account action is needed")
    urgency: str = Field(description="low, medium, or high")


print("Models defined.")

Models defined.


In [14]:
# ──────────────────────────────────────────────
# 2. Define specialist agents (like Rasa forms)
# ──────────────────────────────────────────────

billing_agent = Agent(
    name="Billing Specialist",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    You are a billing support specialist. Have a natural, empathetic conversation
    to understand the customer's billing issue.
    
    Collect through natural conversation:
    - Customer name and email
    - The charge amount in question (if applicable)
    - Description of the billing issue
    - What resolution they want (refund, credit, or investigation)
    - How urgent this is
    
    Ask follow-up questions naturally — don't interrogate. Once you have all info,
    confirm with the customer and then produce your structured output.""",
    handoff_description="Handles billing inquiries, refunds, payment issues, and invoice questions",
    output_type=BillingTicket,
)

technical_agent = Agent(
    name="Technical Support",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    You are a technical support specialist. Have a natural conversation to 
    diagnose and document the customer's technical issue.
    
    Collect through natural conversation:
    - Customer name and email
    - Product/software version
    - What's going wrong (error messages, unexpected behavior)
    - Steps to reproduce (if they can describe them)
    - Urgency level
    
    Be patient and ask clarifying questions. Once you have enough info,
    confirm and produce your structured output.""",
    handoff_description="Handles software bugs, crashes, technical errors, and product issues",
    output_type=TechnicalTicket,
)

account_agent = Agent(
    name="Account Specialist",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    You are an account management specialist. Have a natural conversation
    about the customer's account issue.
    
    Collect through natural conversation:
    - Customer name and email
    - Description of the account problem (locked out, need to change plan, etc.)
    - What action they need taken
    - Urgency level
    
    Be helpful and reassuring. Once you have all info, confirm and produce
    your structured output.""",
    handoff_description="Handles account access, plan changes, cancellations, and profile updates",
    output_type=AccountTicket,
)

print("Specialist agents defined.")

Specialist agents defined.


In [15]:
# ──────────────────────────────────────────────
# 3. Define the triage agent (like Rasa's intent router / stories)
# ──────────────────────────────────────────────

triage_agent = Agent(
    name="Customer Support Triage",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    You are the first point of contact for customer support.
    
    Greet the customer warmly and determine the nature of their request.
    Based on the conversation, hand off to the appropriate specialist:
    
    - Billing issues (charges, refunds, invoices, payments) → Billing Specialist
    - Technical issues (bugs, crashes, errors, product problems) → Technical Support
    - Account issues (access, plan changes, cancellations, profile) → Account Specialist
    
    Ask a clarifying question or two if needed, then hand off. Don't try to solve
    the problem yourself — route to the right specialist.""",
    handoffs=[billing_agent, technical_agent, account_agent],
)

print("Triage agent defined with handoffs to:", [a.name for a in [billing_agent, technical_agent, account_agent]])

Triage agent defined with handoffs to: ['Billing Specialist', 'Technical Support', 'Account Specialist']


### Test 1: Single-shot run (agent handles entire conversation autonomously)

In [16]:
# Single-shot: customer describes everything in one message
# The triage agent routes to the right specialist, who produces structured output

result = await Runner.run(
    triage_agent,
    "Hi, I'm Sarah Chen (sarah@example.com). I've been double-charged $49.99 on my last bill. "
    "This is really urgent — I need a refund as soon as possible.",
)

print(f"Final agent: {result.last_agent.name}")
print(f"Output type: {type(result.final_output)}")
print()

# The output is a structured Pydantic object from whichever specialist handled it
if isinstance(result.final_output, str):
    # Triage agent responded directly (shouldn't happen with good prompting)
    print("Triage response:", result.final_output)
else:
    # Specialist produced structured output
    print(result.final_output.model_dump_json(indent=2))

Final agent: Billing Specialist
Output type: <class '__main__.BillingTicket'>

{
  "customer_name": "Sarah Chen",
  "email": "sarah@example.com",
  "charge_amount": 49.99,
  "billing_issue": "Customer was double-charged $49.99 on her last bill.",
  "resolution": "Refund requested as soon as possible.",
  "urgency": "high"
}


### Test 2: Multi-turn conversation with session memory (most Rasa-like)

In [ ]:
from agents import SQLiteSession

# SQLiteSession automatically persists conversation across turns
# (like Rasa's tracker store)

session = SQLiteSession("customer_session_001", "conversations.db")


async def multi_turn_support():
    """Simulate a multi-turn conversation where info is gathered over time."""

    # Turn 1: Customer initiates vaguely
    print("=" * 60)
    print("TURN 1")
    result = await Runner.run(
        triage_agent,
        "Hey, I need some help with something on my account.",
        session=session,
    )
    print(f"[{result.last_agent.name}]: {result.final_output}")
    print()

    # Turn 2: Customer clarifies it's a technical issue
    print("=" * 60)
    print("TURN 2")
    result = await Runner.run(
        triage_agent,
        "The app keeps crashing when I try to generate reports. I'm on version 4.1.",
        session=session,
    )
    print(f"[{result.last_agent.name}]: {result.final_output}")
    print()

    # Turn 3: Customer provides remaining details
    print("=" * 60)
    print("TURN 3")
    result = await Runner.run(
        triage_agent,
        "I'm Mike Johnson, mike.j@company.com. It happens every time I click Export > PDF. "
        "Started after yesterday's update. This is high priority — my team can't do their weekly reports.",
        session=session,
    )
    print(f"[{result.last_agent.name}]:")
    print(f"Output type: {type(result.final_output)}")

    if not isinstance(result.final_output, str):
        print(result.final_output.model_dump_json(indent=2))
    else:
        print(result.final_output)

    return result


multi_turn_result = await multi_turn_support()

### Test 3: Interactive multi-turn with real user input

In [ ]:
async def interactive_agent_loop():
    """
    Interactive loop using the Agents SDK — type messages, the triage agent
    routes to specialists, and you get a Pydantic object at the end.
    Type 'quit' to exit.
    """
    session = SQLiteSession("interactive_session", "conversations.db")
    current_agent = triage_agent

    print("Customer Support System (type 'quit' to exit)")
    print("=" * 50)

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ("quit", "exit"):
            print("Session ended.")
            return None

        result = await Runner.run(
            current_agent,
            user_input,
            session=session,
        )

        # Track which agent is handling (triage may hand off to specialist)
        current_agent = result.last_agent

        # Check if we got structured output (specialist finished)
        if not isinstance(result.final_output, str):
            print(f"\n[{result.last_agent.name}] produced structured output:")
            print(result.final_output.model_dump_json(indent=2))
            return result.final_output
        else:
            print(f"\n[{result.last_agent.name}]: {result.final_output}")


# Run it
final_output = await interactive_agent_loop()

In [ ]:
# Inspect the final structured output
if final_output:
    print(f"Output type: {type(final_output).__name__}")
    print(final_output.model_dump_json(indent=2))

---
## Rasa vs OpenAI Agents SDK — Comparison

| Feature | Rasa | OpenAI Agents SDK |
|---|---|---|
| **Intent Classification** | NLU pipeline (DIET, etc.) | LLM does it natively |
| **Routing / Stories** | Stories + Rules YAML | Triage Agent + `handoffs=[]` |
| **Form Filling** | Forms + Slots (YAML) | Agent `instructions` + `output_type` |
| **Slot Types** | Predefined (text, bool, float, etc.) | Any Pydantic model |
| **Conversation Memory** | Tracker Store (Redis, SQL, etc.) | `SQLiteSession` (or custom) |
| **Validation** | Slot validation methods | Pydantic validators + Guardrails |
| **Custom Actions** | Action server (Python) | Agent tools (Python functions) |
| **Training Required** | Yes (NLU + stories data) | No — prompt-based |
| **Multi-agent** | Limited | Native handoffs between agents |

### Key Takeaway
The Agents SDK replaces most of Rasa's YAML configuration with Python code and LLM prompts. You get more flexibility and no training data requirements, at the cost of LLM API calls per turn.